In [1]:
import requests
from pathlib import Path
import os

import shelve

from bs4 import BeautifulSoup
from bs4 import Comment
import pandas as pd

from tqdm.auto import tqdm

import math

In [2]:
from diskcache import Cache

cache = Cache("matches_report.shelve")

@cache.memoize()
def get_page(link):
    print(link)
    return requests.get(link).content

In [3]:
len(list(cache))

1704

In [4]:
def build_column_name_prefixes(spans, prefixes):
    assert len(spans) == len(prefixes)
    new_cols = []
    for i in range(len(prefixes)):
        for j in range(spans[i]):
            new_cols.append(prefixes[i])
    return new_cols
            

In [5]:
def join_prefixes_suffixes(prefixes, suffixes):
    assert len(prefixes) == len(suffixes)
    new_names = []
    for i in range(len(prefixes)):
        if prefixes[i] == '':
            new_names.append(suffixes[i])
        else:
            new_names.append(prefixes[i] + '_' + suffixes[i])
    return new_names

In [6]:
leagues = ['EPL', 'Ligue1', 'Bundesliga', 'SerieA', 'LaLiga']
report_types = ['summary', 'passing', 'pass_types', 'defensive_actions', 'possession', 'misc_stats']

In [7]:
for league in leagues:
    print(league)
    path = f'dfs/{league}/matches'
    matches_links = {f.split('.')[0]:os.path.join(path, f) for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))}
    matches_links = dict(sorted(matches_links.items(), key = lambda kv:(kv[1], kv[0]), reverse = True))

    dfs = []
    for season in matches_links:
        df = pd.read_csv(matches_links[season])
        dfs.append(df)

    columns = {}
    datasets = {}

    data = []
    for j in range(6):
        data.append([])

    for j, df in tqdm(enumerate(dfs)):
        if df['Season'].unique()[0] in ['2020-2021', '2019-2020', '2018-2019', '2017-2018']:
            print(df['Season'].unique()[0])
            for i, row in tqdm(df.iterrows()):
                link = row['Match Report']
                if type(link) == str and link.startswith('http'):
                    row_league = row['League']
                    season = row['Season']
                    home_team = row['HomeTeam']
                    away_team = row['AwayTeam']
                    date = row['Date']

                    page = get_page(link)

                    soup = BeautifulSoup(page, 'html.parser')

                    teams = soup.find_all("div", id=lambda value: value and value.startswith("all_player_stats_"))

                    i = -1
                    team_names = [home_team, away_team]

                    for team in teams:
                        i += 1

                        team_name = team_names[i]

                        tables = team.find_all("div", id=lambda value: value and value.startswith("div_stats_"))

                        j = -1
                        for table in tables:
                            j += 1

                            thead_trs = table.find('thead').find_all('tr')

                            col_spans = [1 if el.get('colspan') is None else int(el.get('colspan')) for el in thead_trs[0].find_all('th')]
                            column = [el.text.strip() for el in thead_trs[0].find_all('th')]
                            prefixes = build_column_name_prefixes(col_spans, column)
                            column = [el.text.strip() for el in thead_trs[1].find_all('th')]
                            column = ['Season', 'League', 'Match', 'Team', 'Date'] + join_prefixes_suffixes(prefixes, column)

                            if row_league not in columns:
                                columns[row_league] = {}
                            columns[row_league][j] = column

                            for el in table.find('tbody').find_all('tr'):
                                row = []
                                row.append(season)
                                row.append(row_league)
                                row.append(home_team.replace(' ', '_').strip() + '_vs_' + away_team.replace(' ', '_').strip())
                                row.append(team_name)
                                row.append(date)

                                count_empty = 0
                                for el2 in el.find_all('th'):
                                    row.append(el2.text.strip())
                                    if el2.text.strip() == '':
                                        count_empty += 1
                                for el2 in el.find_all('td'):
                                    row.append(el2.text.strip())
                                    if el2.text.strip() == '':
                                        count_empty += 1
                #                 print(row)
                                if row[-2] == 'On matchday squad, but did not play':
                                    temp = row[-1]
                                    row[-1] = None
                                    for i in range(len(columns[row_league][j]) - len(row) - 1):
                                        row.append(None)
                                    row.append(temp)

                                if count_empty != (len(row) - 5):
                                    data[j].append(row)

    for i in range(6):
        path = Path(f'dfs/{league}/match_reports/')
        path.mkdir(parents=True, exist_ok=True)

        df2 = pd.DataFrame(data[i], columns = columns[f'{row_league}'][i])

        df2.to_csv(os.path.join(path, f'{report_types[i]}.csv'), index = False)


EPL


0it [00:00, ?it/s]

2020-2021


0it [00:00, ?it/s]

https://fbref.com/en/matches/a223dd70/West-Ham-United-Wolverhampton-Wanderers-September-27-2020-Premier-League
https://fbref.com/en/matches/6b258be0/Fulham-Aston-Villa-September-28-2020-Premier-League
https://fbref.com/en/matches/ebe4e309/Liverpool-Arsenal-September-28-2020-Premier-League
https://fbref.com/en/matches/d97aa1b5/Chelsea-Crystal-Palace-October-3-2020-Premier-League
https://fbref.com/en/matches/3cbb397e/Everton-Brighton-and-Hove-Albion-October-3-2020-Premier-League
https://fbref.com/en/matches/5ce15b58/Leeds-United-Manchester-City-October-3-2020-Premier-League
https://fbref.com/en/matches/34ba607f/Newcastle-United-Burnley-October-3-2020-Premier-League
https://fbref.com/en/matches/a29866ec/Leicester-City-West-Ham-United-October-4-2020-Premier-League
https://fbref.com/en/matches/39864083/Southampton-West-Bromwich-Albion-October-4-2020-Premier-League
https://fbref.com/en/matches/09725cb3/Arsenal-Sheffield-United-October-4-2020-Premier-League
https://fbref.com/en/matches/8f9896

...

https://fbref.com/en/matches/038b6a22/Dijon-Monaco-January-26-2019-Ligue-1
https://fbref.com/en/matches/31a6ef67/Guingamp-Reims-January-26-2019-Ligue-1
https://fbref.com/en/matches/d16cad7d/Strasbourg-Bordeaux-January-26-2019-Ligue-1
https://fbref.com/en/matches/03acdfb8/Nice-Nimes-January-26-2019-Ligue-1
https://fbref.com/en/matches/7baec3a3/Montpellier-Caen-January-27-2019-Ligue-1
https://fbref.com/en/matches/c2610f32/Amiens-Lyon-January-27-2019-Ligue-1
https://fbref.com/en/matches/306c9177/Toulouse-Angers-January-27-2019-Ligue-1
https://fbref.com/en/matches/f0a113f8/Paris-Saint-Germain-Rennes-January-27-2019-Ligue-1
https://fbref.com/en/matches/6e3a1e22/Nantes-Saint-Etienne-January-30-2019-Ligue-1
https://fbref.com/en/matches/dd30a3c2/Lille-Nice-February-1-2019-Ligue-1
https://fbref.com/en/matches/34d963b4/Angers-Dijon-February-2-2019-Ligue-1
https://fbref.com/en/matches/0b44ddfa/Rennes-Amiens-February-2-2019-Ligue-1
https://fbref.com/en/matches/07d2b627/Monaco-Toulouse-February-2-2

https://fbref.com/en/matches/1f3d473c/Caen-Angers-April-13-2019-Ligue-1
https://fbref.com/en/matches/e7ff1ff0/Montpellier-Toulouse-April-14-2019-Ligue-1
https://fbref.com/en/matches/ab1e5c68/Rennes-Nice-April-14-2019-Ligue-1
https://fbref.com/en/matches/6a8b98ba/Saint-Etienne-Bordeaux-April-14-2019-Ligue-1
https://fbref.com/en/matches/f0d500da/Lille-Paris-Saint-Germain-April-14-2019-Ligue-1
https://fbref.com/en/matches/9b494820/Nantes-Paris-Saint-Germain-April-17-2019-Ligue-1
https://fbref.com/en/matches/050367b9/Dijon-Rennes-April-19-2019-Ligue-1
https://fbref.com/en/matches/6eea3ae0/Lyon-Angers-April-19-2019-Ligue-1
https://fbref.com/en/matches/43aeb747/Guingamp-Marseille-April-20-2019-Ligue-1
https://fbref.com/en/matches/f2d6766b/Nimes-Bordeaux-April-20-2019-Ligue-1
https://fbref.com/en/matches/3f44f10d/Strasbourg-Montpellier-April-20-2019-Ligue-1
https://fbref.com/en/matches/a0a12160/Nice-Caen-April-20-2019-Ligue-1
https://fbref.com/en/matches/d386d6d6/Toulouse-Lille-April-21-2019-

0it [00:00, ?it/s]

https://fbref.com/en/matches/a68e623d/Monaco-Toulouse-August-4-2017-Ligue-1
https://fbref.com/en/matches/37f2c25f/Paris-Saint-Germain-Amiens-August-5-2017-Ligue-1
https://fbref.com/en/matches/68b9eea2/Saint-Etienne-Nice-August-5-2017-Ligue-1
https://fbref.com/en/matches/b2829d08/Montpellier-Caen-August-5-2017-Ligue-1
https://fbref.com/en/matches/7d2eb66d/Metz-Guingamp-August-5-2017-Ligue-1
https://fbref.com/en/matches/4d28b63b/Lyon-Strasbourg-August-5-2017-Ligue-1
https://fbref.com/en/matches/bf730e30/Troyes-Rennes-August-5-2017-Ligue-1
https://fbref.com/en/matches/a5686dbd/Lille-Nantes-August-6-2017-Ligue-1
https://fbref.com/en/matches/f85b1b75/Angers-Bordeaux-August-6-2017-Ligue-1
https://fbref.com/en/matches/ab13ffd4/Marseille-Dijon-August-6-2017-Ligue-1
https://fbref.com/en/matches/7faa9279/Nice-Troyes-August-11-2017-Ligue-1
https://fbref.com/en/matches/52194605/Rennes-Lyon-August-11-2017-Ligue-1
https://fbref.com/en/matches/1ad6b969/Nantes-Marseille-August-12-2017-Ligue-1
https://

KeyboardInterrupt: 

In [10]:
list(cache)[0][1]

'https://fbref.com/en/matches/6c2be382/Atalanta-Chievo-February-4-2018-Serie-A'

In [25]:
link = list(cache)[0][1]
page = get_page(link)

soup = BeautifulSoup(page, 'html.parser')

teams = soup.find_all("div", id=lambda value: value and value.startswith("all_player_stats_"))

doc_columns = []
doc_meanings = []

i = -1
team_names = ['Atalanta', 'Chievo']

for team in teams:
    i += 1

    team_name = team_names[i]

    tables = team.find_all("div", id=lambda value: value and value.startswith("div_stats_"))

    j = -1
    for table in tables:
        j += 1

        thead_trs = table.find('thead').find_all('tr')

        col_spans = [1 if el.get('colspan') is None else int(el.get('colspan')) for el in thead_trs[0].find_all('th')]
        column = [el.text.strip() for el in thead_trs[0].find_all('th')]
        prefixes = build_column_name_prefixes(col_spans, column)
        column = [el.text.strip() for el in thead_trs[1].find_all('th')]
        meanings = ['Season', 'League', 'Team', 'Date'] + [BeautifulSoup(el.get('data-tip').replace('<br>', '; '), "lxml").text if el.has_attr('data-tip') else el.text.strip() for el in thead_trs[1].find_all('th')]
        column = ['Season', 'League', 'Team', 'Date'] + join_prefixes_suffixes(prefixes, column)
        k = 0
        for col in column:
            if col not in doc_columns:
                doc_columns.append(col.replace(" ", '_'))
                doc_meanings.append(meanings[k])
            k += 1
        
        print(len(meanings), len(column))
    
    break

36 36
31 31
35 35
33 33
34 34
26 26


In [28]:
import json
json = json.dumps(dict(zip(doc_columns, doc_meanings)))
f = open("dfs/player_match_reports_column_documentation.json", "w")
f.write(json)
f.close()

In [29]:
dict(zip(doc_columns, doc_meanings))

{'Season': 'Season',
 'League': 'League',
 'Team': 'Team',
 'Date': 'Date',
 'Player': 'Player',
 '#': 'Shirt Number',
 'Nation': 'Nationality of the player.; First, we check our records in international play at senior level.; Then youth level.; Then citizenship presented on wikipedia.; Finally, we use their birthplace when available.',
 'Pos': 'GK - Goalkeepers; DF - Defenders; MF - Midfielders; FW - Forwards; FB - Fullbacks; LB - Left Backs; RB - Right Backs; CB - Center Backs; DM - Defensive Midfielders; CM - Central Midfielders; LM - Left Midfielders; RM - Right Midfielders; WM - Wide Midfielders; LW - Left Wingers; RW - Right Wingers; AM - Attacking Midfielders; ',
 'Age': "Age on date of match; Age is shown in YY-DDD format; Note: Ages are based on the player's age relative to UTC when the web page is first created. This may cause some issues where a player's age differs by one day from what you might expect",
 'Min': 'Minutes',
 'Performance_Gls': 'Goals; Goals scored or allowed